In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
# Read the data
data = pd.read_csv('Pokemon.csv', index_col='#')

In [11]:
data.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [12]:
data2 = data.rename({'Type 1': 'Type1'}, axis=1)

In [13]:
def score(x):
    if x  == 'Bug':
        return 0
    if x == 'Dark':
        return 1
    if x == 'Dragon':
        return 2
    if x == 'Electric':
        return 3
    if x == 'Fairy':
        return 4
    if x == 'Fighting':
        return 5
    if x == 'Fire':
        return 6
    if x == 'Flying':
        return 7
    if x == 'Ghost':
        return 8
    if x == 'Grass':
        return 9
    if x == 'Ground':
        return 10
    if x == 'Ice':
        return 11
    if x == 'Normal':
        return 12
    if x == 'Poison':
        return 13
    if x == 'Psychic':
        return 14
    if x == 'Rock':
        return 15
    if x == 'Steel':
        return 16
    if x == 'Water':
        return 17

In [14]:
data2['Type1_score'] = data2['Type1'].apply(score)

# NEW Method

In [15]:
cols_to_use = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation']
X = data2[cols_to_use]

In [16]:
# Remove rows with missing target, separate target from predictors
data2.dropna(axis=0, subset=['Type1_score'], inplace=True)
y = data2.Type1_score         
data2.drop(['Type1_score'], axis=1, inplace=True)

In [17]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [18]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

In [19]:
# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [20]:
# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = data2[my_cols].copy()

In [21]:
# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [22]:
from xgboost import XGBRegressor

In [23]:
# Define the model
my_model = XGBRegressor(random_state=0)

In [24]:
# Fit the model
my_model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
from sklearn.metrics import mean_absolute_error

In [27]:
# Get predictions
predictions_1 = my_model.predict(X_valid)

In [28]:
mae_1 = mean_absolute_error(predictions_1, y_valid)

In [29]:
# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 4.847119061928242


In [47]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.5)

# Fit the model
my_model_2.fit(X_train, y_train)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)

print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 4.980402687564492


In [42]:
#### Define the model
my_model_3 = XGBRegressor(n_estimators=1)

# Fit the model
my_model_3.fit(X_train, y_train)

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)

Mean Absolute Error: 7.0233860775828365
